In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, RidgeCV, Lasso, \
LassoCV, ElasticNet, ElasticNetCV
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.decomposition import PCA

# 2022

In [2]:
team_stats = pd.read_csv('../footy_data/league_one_teams_2022.csv')
games = pd.read_csv('../footy_data/league_one_games_2022.csv')

In [3]:
games = games[0:62]
games
#Only include completed games

,timestamp,date_GMT,status,attendance,home_team_name,away_team_name,referee,Game Week,Pre-Match PPG (Home),Pre-Match PPG (Away),...,odds_ft_home_team_win,odds_ft_draw,odds_ft_away_team_win,odds_ft_over15,odds_ft_over25,odds_ft_over35,odds_ft_over45,odds_btts_yes,odds_btts_no,stadium_name
0,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Bolton Wanderers,Milton Keynes Dons,Martin Coy,1,0.0,0.0,...,2.45,3.35,2.75,1.29,1.80,2.80,4.95,1.69,2.15,University of Bolton Stadium (Bolton)
1,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Cambridge United,Oxford United,David Rock,1,0.0,0.0,...,3.55,3.40,2.05,1.35,1.95,3.10,5.50,1.80,2.00,Abbey Stadium (Cambridge- Cambridgeshire)
2,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Crewe Alexandra,Cheltenham Town,Ben Toner,1,0.0,0.0,...,2.25,3.05,3.40,1.39,2.10,3.40,6.75,1.87,1.91,Mornflake Stadium (Crewe- Cheshire)
3,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Doncaster Rovers,AFC Wimbledon,Samuel Barrott,1,0.0,0.0,...,2.40,3.25,2.95,1.34,1.95,3.10,5.75,1.77,2.00,The Keepmoat Stadium (Doncaster- South Yorkshire)
4,1628344800,Aug 07 2021 - 2:00pm,complete,NaN,Fleetwood Town,Portsmouth,Ben Speedie,1,0.0,0.0,...,2.95,3.15,2.40,1.48,2.25,3.90,7.50,2.05,1.74,Highbury Stadium (Fleetwood- Lancashire)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,1630159200,Aug 28 2021 - 2:00pm,complete,NaN,Shrewsbury Town,Gillingham,Samuel Allison,5,0.0,0.5,...,2.30,3.20,3.05,1.48,2.30,4.00,7.75,2.10,1.74,Montgomery Waters Meadow (Shrewsbury- Shropshire)
58,1630159200,Aug 28 2021 - 2:00pm,complete,NaN,Sunderland,Wycombe Wanderers,John Busby,5,3.0,2.0,...,2.00,3.35,3.65,1.37,2.00,3.30,6.50,1.91,1.87,Stadium of Light (Sunderland)
59,1630159200,Aug 28 2021 - 2:00pm,complete,NaN,Wigan Athletic,Portsmouth,Chris Sarginson,5,2.0,2.0,...,2.40,3.15,3.00,1.41,2.15,3.55,7.00,1.91,1.87,DW Stadium (Wigan)
60,1630764000,Sep 04 2021 - 2:00pm,complete,NaN,AFC Wimbledon,Oxford United,Brett Huxtable,6,1.0,0.5,...,2.95,3.30,2.35,1.30,1.80,2.90,5.25,1.69,2.15,Plough Lane (London)


Seeing how many games had goals and xg of over 2.5

In [4]:
games['over/under'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 2.5, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['xg_over/under'] = np.where(games['team_a_xg'] + games['team_b_xg'] > 2.99, 1, 0)
# Adding column 0 represents under 2.5, 1 represents over 2.5
games['over/under_1.5'] = np.where(games['home_team_goal_count'] + games['away_team_goal_count'] > 1.5, 1, 0)
# Adding column 0 represents under 1.5, 1 represents over 1.5

In [5]:
season_goal_total = games['total_goal_count'].sum()
season_1h_goal_total = games['total_goals_at_half_time'].sum()
percent_goals_1h = ((season_1h_goal_total)/(season_goal_total)) * 100
percent_goals_1h

45.07042253521127

In [6]:
games['over/under'].value_counts()

0    37
1    25
Name: over/under, dtype: int64

In [7]:
games['xg_over/under'].value_counts()

0    45
1    17
Name: xg_over/under, dtype: int64

In [8]:
xg_view = games[['home_team_name', 'away_team_name', 'team_a_xg', 'team_b_xg', 'home_team_goal_count', 'away_team_goal_count']]

In [9]:
xg_view[5:63]

,home_team_name,away_team_name,team_a_xg,team_b_xg,home_team_goal_count,away_team_goal_count
5,Gillingham,Lincoln City,1.90,1.07,1,1
6,Ipswich Town,Morecambe,1.41,0.95,2,2
7,Rotherham United,Plymouth Argyle,2.05,0.95,2,0
8,Shrewsbury Town,Burton Albion,1.50,1.65,0,1
9,Wycombe Wanderers,Accrington Stanley,1.70,1.11,2,1
10,Sunderland,Wigan Athletic,1.69,1.03,2,1
11,Charlton Athletic,Sheffield Wednesday,1.02,0.88,0,0
12,AFC Wimbledon,Bolton Wanderers,1.81,1.18,3,3
13,Accrington Stanley,Cambridge United,1.78,1.33,2,1
14,Burton Albion,Ipswich Town,1.30,1.26,2,1


In [10]:
first_matchweek = games[0:12]

In [11]:
first_matchweek['over/under'].value_counts()

0    7
1    5
Name: over/under, dtype: int64

In [12]:
first_matchweek['over/under_1.5'].value_counts()

1    9
0    3
Name: over/under_1.5, dtype: int64

In [13]:
games['over/under_1.5'].value_counts()

1    42
0    20
Name: over/under_1.5, dtype: int64

In [14]:
games['over/under'].value_counts()

0    37
1    25
Name: over/under, dtype: int64

In [15]:
games['total_goals_at_half_time'].value_counts()

0    21
1    20
2    19
3     2
Name: total_goals_at_half_time, dtype: int64

In [16]:
games['home_team_goal_count_half_time'].sum()

39

In [17]:
games['away_team_goal_count_half_time'].sum()

25

# Teams

In [18]:
team_stats[['team_name', 'xg_for_avg_overall', 'xg_against_avg_overall']]

,team_name,xg_for_avg_overall,xg_against_avg_overall
0,Sunderland AFC,1.29,1.02
1,Burton Albion FC,1.19,1.37
2,Rotherham United FC,1.35,0.55
3,Sheffield Wednesday FC,0.76,0.77
4,Ipswich Town FC,1.05,1.00
5,Wigan Athletic FC,0.71,0.96
6,Charlton Athletic FC,0.96,0.99
7,Bolton Wanderers FC,1.54,1.14
8,Milton Keynes Dons FC,1.44,0.97
9,Doncaster Rovers FC,0.53,1.22


sos = total average of xg for xg against of opponents they have played